In [1]:
import sqlite3
from sqlite3 import Error
import numpy as np
import pandas as pd

### Загрузим датасет с ирисами в формате .csv, найденный на GitHub 

In [4]:
import pandas as pd
irisData = pd.read_csv('https://github.com/Muhd-Shahid/Write-Raw-File-into-Database-Server/raw/main/iris.csv',index_col=False)
irisData

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [5]:
#Создаем базу данных на компьютере
try:
    sqlite_connection = sqlite3.connect('sqlite_python.db')
    cursor = sqlite_connection.cursor()
    print("База данных создана и успешно подключена к SQLite")

    sqlite_select_query = "select sqlite_version();"
    cursor.execute(sqlite_select_query)
    record = cursor.fetchall()
    print("Версия базы данных SQLite: ", record)
    cursor.close()

except sqlite3.Error as error:
    print("Ошибка при подключении к sqlite", error)
finally:
    if (sqlite_connection):
        sqlite_connection.close()
        print("Соединение с SQLite закрыто")

База данных создана и успешно подключена к SQLite
Версия базы данных SQLite:  [('3.36.0',)]
Соединение с SQLite закрыто


In [6]:
#Создаем подключение к созданной базе
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path,timeout=10)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [7]:
connection = create_connection("sqlite_python.db")

Connection to SQLite DB successful


In [9]:
#Создаем функцию для создания новых таблиц и добавления записей
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
    finally:
        if (sqlite_connection):
            sqlite_connection.close()
            print("Соединение с SQLite закрыто")

In [101]:
table_iris = '''CREATE TABLE IF NOT EXISTS iris_2 (
    iris_id INTEGER PRIMARY KEY,
    sepal_length FLOAT(2,1) NOT NULL, 
    sepal_width FLOAT(2,1) NOT NULL,
    petal_length FLOAT(2,1) NOT NULL,
    petal_width FLOAT(2,1) NOT NULL,
    species CHAR(11)NOT NULL
);'''
execute_query(connection,table_iris)

Query executed successfully
Соединение с SQLite закрыто


In [82]:
#Создаем функцию для получение выборок из базы с помощью SQL сразу в табличной форме
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return pd.DataFrame(result,columns=[member[0] for member in cursor.description])
    except Error as e:
        print(f"The error '{e}' occurred")

    finally:
        if (sqlite_connection):
            sqlite_connection.close()
            print("Соединение с SQLite закрыто")

In [106]:
#Загружаем строки из датасета ирисов в таблицу
for i,row in irisData.iterrows():
    iris_to_sql = '''
    INSERT INTO iris_2(sepal_length, sepal_width, petal_length, petal_width, species) 
    VALUES (?,?,?,?,?)
    '''
    cursor = connection.cursor()
    cursor.execute(iris_to_sql, (tuple(row)))
    sqlite_connection.close()

### Выведем первые 10 строк

In [105]:
execute_read_query(connection,'''
select * from iris_2
limit 10
''')

Соединение с SQLite закрыто


,iris_id,sepal_length,sepal_width,petal_length,petal_width,species
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa
5,6,5.4,3.9,1.7,0.4,setosa
6,7,4.6,3.4,1.4,0.3,setosa
7,8,5.0,3.4,1.5,0.2,setosa
8,9,4.4,2.9,1.4,0.2,setosa
9,10,4.9,3.1,1.5,0.1,setosa
